In [1]:
import os 
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold, SelectKBest, chi2, f_regression, SelectFromModel, mutual_info_regression
from scipy.stats import f_oneway
from scipy.sparse import csr_matrix
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVR
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import MinMaxScaler, StandardScaler 
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import r2_score
from sklearn.model_selection import RandomizedSearchCV
from category_encoders import BinaryEncoder
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.one_hot import OneHotEncoder
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
os.chdir('..')

In [3]:
df = pd.read_csv('ENG_DATA/CLEANED/12-8_Cleaned_df.csv', index_col = [0])

# Fix Missing Data

In [4]:
#fix a bug 
for l in ['charter', 'private', 'public', 'gsRating', 'enrollment']: 
    df[l].fillna(df[l].mean(), inplace=True)

# Add Dates

In [5]:
df["on_market_date"] = df["on_market_date"].apply(lambda x: datetime.strptime(x, "%m/%d/%y"))
df["sale_date"] = df["sale_date"].apply(lambda x: datetime.strptime(x, "%m/%d/%y"))

In [6]:
df["on_market_month_year"] = df["on_market_date"].apply(lambda x: x.strftime("%m/%Y"))
df["sale_month_year"] = df["sale_date"].apply(lambda x: x.strftime("%m/%Y"))

# Functions to Test

In [7]:
lr = LinearRegression(normalize=True)

In [8]:
def get_score(model, x, y):
    # Split dataset into train and validation subsets:
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = r2_score(y_test, y_pred)
    return score
def get_coef(model, x, y): 
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
    model.fit(X_train, y_train)
    return pd.DataFrame(model.coef_, index = X_train.columns, columns=['coef']).sort_values('coef',ascending=False)

In [69]:
location_cols = ['longitude', 'latitude', 'elevation', 'street_name', 'zip', 'area', 'street_name', 'zoning', 'neighborhood', 'views', 'shopping']

In [70]:
x = df.drop(columns = ['sale_price', 'orig_list_price', 'city', 'full_address', 'park_leased'])
x.drop(location_cols, axis=1, inplace=True)
y = df.sale_price
x_num = x.select_dtypes(include = 'number')
x_cat = x.select_dtypes(exclude = 'number')

# Feature Highlighting & Baseline

In [71]:
x_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23720 entries, 0 to 23719
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   on_market_date        23720 non-null  datetime64[ns]
 1   sale_date             23720 non-null  datetime64[ns]
 2   drive_side            23720 non-null  object        
 3   parking               23720 non-null  object        
 4   transportation        23720 non-null  object        
 5   type                  23720 non-null  object        
 6   on_market_month_year  23720 non-null  object        
 7   sale_month_year       23720 non-null  object        
dtypes: datetime64[ns](2), object(6)
memory usage: 1.6+ MB


In [72]:
x_cat.nunique()

on_market_date          3396
sale_date               2655
drive_side                16
parking                  152
transportation            16
type                     259
on_market_month_year     133
sale_month_year          120
dtype: int64

In [73]:
x_cat.head(5)

,on_market_date,sale_date,drive_side,parking,transportation,type,on_market_month_year,sale_month_year
0,2013-03-14,2013-03-22,"PVDW,PVSW","ATCH,GARG",1BLK,3STR,03/2013,03/2013
1,2017-05-18,2017-08-17,0,"ATCH,GARG,ATDR,INAC",1BLK,0,05/2017,08/2017
2,2010-06-27,2010-08-20,"PVDW,PVSW",GARG,1BLK,"ATAC,2STR,FIXR",06/2010,08/2010
3,2012-06-07,2012-07-13,PVDW,"ATCH,GARG,ATDR,INAC",2BLK,3STR,06/2012,07/2012
4,2018-05-17,2018-06-21,0,"ATCH,GARG,ATDR,INAC",1BLK,2STR,05/2018,06/2018


In [74]:
%%time
baseline_lr_score = get_score(lr, x.select_dtypes(include = 'number'), y)
print('Linear Regression score without feature engineering:', baseline_lr_score)

Linear Regression score without feature engineering: 0.6319655908401001
CPU times: user 109 ms, sys: 26.7 ms, total: 136 ms
Wall time: 137 ms


In [75]:
get_coef(lr, x_num, y).head(10)

,coef
trea,4.685570e+14
pornography/obscene mat,4.685570e+14
suicide,4.685570e+14
stolen property,4.685570e+14
disorderly conduct,4.685570e+14
extortion,4.685570e+14
kidnapping,4.685570e+14
bribery,4.685570e+14
arson,4.685570e+14
bad checks,4.685570e+14


# Feature Elimination
1. Drop Highly Correlated Features

In [76]:
# FROM: https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/
corr_matrix = x_num.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]

In [77]:
x_fe = x.drop(x[to_drop], axis=1).copy()
print("LR r2_score: {:.3f}".format(get_score(lr, x_fe.select_dtypes(include = 'number'), y)))

LR r2_score: 0.602


# Linear Regression Transformation 
1. OneHot Encode variables with comma-seperated features
2. BinaryEncoder for sparse features
3. Ordinal Encoding for Month-Year and Neighborhood

In [78]:
x_lr = x_fe.copy()

In [79]:
x_lr.drop(columns = ['on_market_date'], inplace=True)

In [80]:
x_lr = x_lr.merge(x_lr['parking'].str.get_dummies(sep=',').rename(lambda x: 'parking_' + x, axis='columns'), left_index=True, right_index=True)

In [81]:
x_lr = x_lr.merge(x_lr['drive_side'].str.get_dummies(sep=',').rename(lambda x: 'drive_side_' + x, axis='columns'), left_index=True, right_index=True)

In [82]:
x_lr = x_lr.merge(x_lr['type'].str.get_dummies(sep=',').rename(lambda x: 'type_' + x, axis='columns'), left_index=True, right_index=True)

In [83]:
x_lr.drop(columns = ['parking', 'drive_side', 'type'], inplace=True)

In [84]:
x_lr.select_dtypes(exclude = 'number').nunique()

sale_date               2655
transportation            16
on_market_month_year     133
sale_month_year          120
dtype: int64

In [85]:
%%time
dummy1_lr_score = get_score(lr, x_lr.select_dtypes(include = 'number'), y)
print('Linear Regression score with 1-hot encoding:', dummy1_lr_score)

Linear Regression score with 1-hot encoding: 0.6191789317059218
CPU times: user 112 ms, sys: 30.4 ms, total: 142 ms
Wall time: 115 ms


## Binary Encoding

In [86]:
# ce_bin = BinaryEncoder(cols = ['street_name'])
ce_bin = BinaryEncoder(cols = ['transportation'])
x_lr2 = ce_bin.fit_transform(x_lr)

In [87]:
%%time
dummy2_lr_score = get_score(lr, x_lr2.select_dtypes(include = 'number'), y)
print('Linear Regression score with target encoding:', dummy2_lr_score)

Linear Regression score with target encoding: 0.6197382506377681
CPU times: user 111 ms, sys: 20.2 ms, total: 131 ms
Wall time: 100 ms


## Ordinal Encoding

In [88]:
ce_oe = OrdinalEncoder(cols = ['on_market_month_year', 'sale_month_year'])
x_lr3 = ce_oe.fit_transform(x_lr2, y)

In [89]:
%%time
dummy3_lr_score = get_score(lr, x_lr3.select_dtypes(include = 'number'), y)
print('Linear Regression score with ordinal encoding:', dummy3_lr_score)

Linear Regression score with ordinal encoding: 0.6200124779695756
CPU times: user 121 ms, sys: 31.1 ms, total: 152 ms
Wall time: 119 ms


In [90]:
get_coef(lr, x_lr3.select_dtypes(include = 'number'), y)

,coef
type_4STR,1.550833e+06
baths,4.084683e+05
drive_side_SHDW,3.081843e+05
parking_GSPC,1.480289e+05
num_parking,1.418776e+05
...,...
drive_side_DIRT,-1.880935e+05
pornography/obscene mat,-2.141571e+05
drive_side_GRVL,-2.169495e+05
type_TWNH,-3.266633e+05


# Adding Location Variables

In [ ]:
x_lc = x.copy()

In [ ]:
xy_scaler = std_scaler.fit(x_lc[["latitude","longitude"]])
x_lc[["latitude","longitude"]]=xy_scaler.transform(x_lc[["latitude","longitude"]])

In [ ]:
x_lc["rot45latitude"] = .707* x_lc["longitude"] + .707* x_lc["latitude"] 
x_lc["rot45longitude"] = .707* x_lc["longitude"] - .707* x_lc["latitude"]
x_lc["rot30latitude"] = (1.732/2)* x_lc["latitude"] + (1./2)* x_lc["longitude"] 
x_lc["rot30longitude"] = (1.732/2)* x_lc["longitude"] - (1./2)* x_lc["latitude"]
x_lc["rot60latitude"] = (1./2)* x_lc["latitude"] + (1.732/2)* x_lc["longitude"] 
x_lc["rot60longitude"] = (1./2)* x_lc["longitude"] - (1.732/2)* x_lc["latitude"]
x_lc["radial_r"] = np.sqrt( np.power(x_lc["longitude"],2) + np.power(x_lc["latitude"],2))

In [ ]:
print("LR r2_score: {:.3f}".format(get_score(rf, x_lc.select_dtypes(include = 'number'), y)))

# Adding Square Variables

In [102]:
x_lr4 = x_lr3.copy()

In [103]:
square_cols_list = []
cols_num = list(set(list(x_lr4.columns)) - set(list(x_lr3.columns)).difference(set(list(x_num.columns))))
for col in cols_num: 
    square_cols_list.append(col + "_2") 

In [105]:
x_lr4[square_cols_list] = x_lr4[cols_num].pow(2)

In [97]:
print("LR r2_score: {:.3f}".format(get_score(lr, x_lr4.select_dtypes(include = 'number'), y)))

LR r2_score: 0.696


In [100]:
get_coef(lr, x_lr4.select_dtypes(include = 'number'), y)

,coef
pornography/obscene mat,1.360621e+07
type_4STR,1.165367e+06
extortion,3.018293e+05
drive_side_SHDW,2.739613e+05
gsRating,2.703085e+05
...,...
charter,-2.327292e+05
transportation_0,-2.345133e+05
public,-2.659102e+05
pornography/obscene mat_2,-1.395441e+07


# Eliminate Features

In [108]:
corr_matrix = x_lr4.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [109]:
x_lr5 = x_lr4.drop(x_lr4[to_drop], axis=1).copy()
print("LR r2_score: {:.3f}".format(get_score(lr, x_lr5.select_dtypes(include = 'number'), y)))

LR r2_score: 0.685


In [110]:
x_lr5.columns.shape[0]

98

# Recursive Feature Elimination

In [ ]:
# classifications
rfecv = RFECV(lr, step=5, min_features_to_select = 60,
              scoring='r2')
rfecv.fit(x_lr5, y)
print("Optimal number of features : %d" % rfecv.n_features_)

In [ ]:
# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(50, 5*len(rfecv.grid_scores_) + 50, 5), rfecv.grid_scores_)
plt.show()

In [ ]:
x_lr_rfe = x_lr5[x_lr5.columns[rfecv.support_]]

In [ ]:
print("LR r2_score: {:.3f}".format(get_score(lr, x_lr_rfe, y)))

# Export

In [111]:
def export_df(df_, name_):
    month_day = datetime.now().strftime('%m-%d')
    df_.to_csv('ENG_DATA/SELECTED/{}_{}.csv'.format(month_day, name_))

In [113]:
export_df(x_lr4.merge(y, left_index = True, right_index = True), "lr_noloc")